# TransformersのQuestion Answeringを用いた係り受け解析器
## モデル簡易作成ツール(fastトークナイザ向け)


必要なパッケージと各conlluを準備

In [ ]:
import os
url="https://github.com/UniversalDependencies/UD_Japanese-GSDLUW"
d=os.path.basename(url)
!test -d $d || git clone --depth=1 $url
!for F in train dev test ; do cat $d/*-$$F.conllu > $$F.conllu ; done
url="https://github.com/huggingface/transformers"
d=os.path.basename(url)
!pip install $d pytokenizations ufal.chu-liu-edmonds datasets seqeval
!test -d $d || git clone --depth=1 -b `pip list | sed -n "s/^$d */v/p"` $url
url="https://universaldependencies.org/conll18/conll18_ud_eval.py"
f=os.path.basename(url)
!test -f $f || curl -LO $url

qa-model作成(HEAD返答モデル)

In [ ]:
import tokenizations,json
from transformers import AutoTokenizer
src="KoichiYasuoka/deberta-base-japanese-aozora"
m,n=AutoTokenizer.from_pretrained(src).mask_token,0
for f in ["train","dev","test"]:
  u,v,w=[],None,[]
  with open(f+".conllu","r",encoding="utf-8") as r:
    for s in r.read().split("\n"):
      if s.startswith("# text = "):
        v=s[9:]
      elif s=="":
        if v and w!=[]:
          v2w,w2v=tokenizations.get_alignments(v,[x[1] for x in w])
          for i,t in enumerate([] if (w2v+[[]]).index([])<len(w2v) else w):
            n,c,h=n+1,v[0:w2v[i][0]]+m+v[w2v[i][-1]+1:],int(t[6])-1
            s=w2v[i if h<0 else h][0]+(len(m)-len(t[1]) if i<h else 0)
            a={"text":[m if h<0 else w[h][1]],"answer_start":[s]}
            u.append({"id":n,"context":c,"question":t[1],"answers":a})
            if f=="train":
              if [1 for x in w if x[1]==t[1]]==[1]:
                n,j=n+1,i if h<0 else h
                a={"text":[w[j][1]],"answer_start":[w2v[j][0]]}
                u.append({"id":n,"context":v,"question":t[1],"answers":a})
        v,w=None,[]
      else:
        t=s.split("\t")
        if len(t)==10 and t[0].isdecimal():
          w.append(t)
  with open(f+".json","w",encoding="utf-8") as w:
    json.dump({"data":u},w,ensure_ascii=False,indent=2)
s=" ".join(["transformers/examples/pytorch/question-answering/run_qa.py",
  "--model_name_or_path="+src,
  "--output_dir=qa-model --overwrite_output_dir --save_total_limit=2",
  "--per_device_train_batch_size=16 --per_device_eval_batch_size=16",
  "--train_file=train.json --validation_file=dev.json --test_file=test.json",
  "--do_train --do_eval --do_predict --evaluation_strategy=epoch",
  "--learning_rate=5e-05 --num_train_epochs=3 --warmup_ratio=0.1"])
!python $s

qa-model/deprel作成(DEPREL付与モデル)

In [ ]:
import json
v,y=set(),set()
for f in ["test","dev","train"]:
  u=[]
  with open(f+".conllu","r",encoding="utf-8") as r:
    for s in r.read().split("\n\n"):
      a=[x.split("\t") for x in s.split("\n")]
      b=[x for x in a if len(x)==10 and x[7]!="_"]
      if b!=[]:
        t,p=[x[1] for x in b],["B-"+x[7] for x in b]
        u.append({"tokens":t,"pos":p})
        if f=="train":
          v-=set(p)
        else:
          v|=set(p)
        y|=set([j for i,j in zip(t,p) if len(i)>1])
  if f=="train":
    u.append({"tokens":["〓"]*(len(v|y)),"pos":list(v)+["I-"+x[2:] for x in y]})
  with open(f+".json","w",encoding="utf-8") as w:
    for t in u:
      print(json.dumps(t,ensure_ascii=False),file=w)
s=" ".join(["transformers/examples/pytorch/token-classification/run_ner.py",
  "--model_name_or_path=qa-model --task_name=pos --label_all_tokens",
  "--output_dir=qa-model/deprel --overwrite_output_dir --save_total_limit=2",
  "--train_file=train.json --validation_file=dev.json --test_file=test.json",
  "--do_train --do_eval --do_predict --evaluation_strategy=epoch",
  "--learning_rate=5e-05 --num_train_epochs=3 --warmup_ratio=0.1"])
!python $s

qa-model/tagger作成(UPOS・FEATS付与モデル)

In [ ]:
import json
v,y=set(),set()
for f in ["test","dev","train"]:
  u=[]
  with open(f+".conllu","r",encoding="utf-8") as r:
    for s in r.read().split("\n\n"):
      a=[x.split("\t") for x in s.split("\n")]
      b=[x for x in a if len(x)==10 and x[3]!="_"]
      if b!=[]:
        t,p=[x[1] for x in b],["B-"+x[3]+"|"+x[5] for x in b]
        u.append({"tokens":t,"pos":p})
        if f=="train":
          v-=set(p)
        else:
          v|=set(p)
        y|=set([j for i,j in zip(t,p) if len(i)>1])
  if f=="train":
    u.append({"tokens":["〓"]*(len(v|y)),"pos":list(v)+["I-"+x[2:] for x in y]})
  with open(f+".json","w",encoding="utf-8") as w:
    for t in u:
      print(json.dumps(t,ensure_ascii=False),file=w)
s=" ".join(["transformers/examples/pytorch/token-classification/run_ner.py",
  "--model_name_or_path=qa-model --task_name=pos --label_all_tokens",
  "--output_dir=qa-model/tagger --overwrite_output_dir --save_total_limit=2",
  "--train_file=train.json --validation_file=dev.json --test_file=test.json",
  "--do_train --do_eval --do_predict --evaluation_strategy=epoch",
  "--learning_rate=5e-05 --num_train_epochs=3 --warmup_ratio=0.1"])
!python $s

評価とテスト

In [ ]:
class TransformersUD(object):
  def __init__(self,bert):
    import os
    from transformers import (AutoTokenizer,AutoModelForQuestionAnswering,
      AutoModelForTokenClassification,AutoConfig,TokenClassificationPipeline)
    self.tokenizer=AutoTokenizer.from_pretrained(bert)
    self.model=AutoModelForQuestionAnswering.from_pretrained(bert)
    x=AutoModelForTokenClassification.from_pretrained
    if os.path.isdir(bert):
      d,t=x(os.path.join(bert,"deprel")),x(os.path.join(bert,"tagger"))
    else:
      from transformers.file_utils import hf_bucket_url
      c=AutoConfig.from_pretrained(hf_bucket_url(bert,"deprel/config.json"))
      d=x(hf_bucket_url(bert,"deprel/pytorch_model.bin"),config=c)
      s=AutoConfig.from_pretrained(hf_bucket_url(bert,"tagger/config.json"))
      t=x(hf_bucket_url(bert,"tagger/pytorch_model.bin"),config=s)
    self.deprel=TokenClassificationPipeline(model=d,tokenizer=self.tokenizer,
      aggregation_strategy="simple")
    self.tagger=TokenClassificationPipeline(model=t,tokenizer=self.tokenizer)
  def __call__(self,text):
    import numpy,torch,ufal.chu_liu_edmonds
    w=[(t["start"],t["end"],t["entity_group"]) for t in self.deprel(text)]
    z,n={t["start"]:t["entity"].split("|") for t in self.tagger(text)},len(w)
    r,m=[text[s:e] for s,e,p in w],numpy.full((n+1,n+1),numpy.nan)
    v=self.tokenizer(r,add_special_tokens=False)["input_ids"]
    for i,t in enumerate(v):
      q=[self.tokenizer.cls_token_id]+t+[self.tokenizer.sep_token_id]
      c=[q]+v[0:i]+[[self.tokenizer.mask_token_id]]+v[i+1:]+[[q[-1]]]
      b=[len(sum(c[0:j+1],[])) for j in range(len(c))]
      d=self.model(input_ids=torch.tensor([sum(c,[])]),
        token_type_ids=torch.tensor([[0]*b[0]+[1]*(b[-1]-b[0])]))
      s,e=d.start_logits.tolist()[0],d.end_logits.tolist()[0]
      for j in range(n):
        m[i+1,0 if i==j else j+1]=s[b[j]]+e[b[j+1]-1]
    h=ufal.chu_liu_edmonds.chu_liu_edmonds(m)[0]
    u="# text = "+text.replace("\n"," ")+"\n"
    for i,(s,e,p) in enumerate(w,1):
      u+="\t".join([str(i),r[i-1],"_",z[s][0][2:],"_","|".join(z[s][1:]),
        str(h[i]),p,"_","_" if i<n and w[i][0]<e else "SpaceAfter=No"])+"\n"
    return u+"\n"

nlp=TransformersUD("qa-model")
for f in ["dev.conllu","test.conllu"]:
  with open(f,"r",encoding="utf-8") as r:
    with open("result-"+f,"w",encoding="utf-8") as w:
      for s in r.read().split("\n"):
        if s.startswith("# text = "):
          w.write(nlp(s[9:]))
print("*** dev.conllu")
!python conll18_ud_eval.py -v dev.conllu result-dev.conllu
print("*** test.conllu")
!python conll18_ud_eval.py -v test.conllu result-test.conllu